## 每次只修复一个线圈

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.lines as mlines
import random
import os
from impyute import locf
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
# %%
# correlation  both
from sklearn.linear_model import LinearRegression


# lanes = [1,2,3,4]

lidu = 5
startday = 4
endday = 24
maindet = ['ZHWX33','ZHWX34','ZHWX35']
# 数据准备
cmap = cm.tab10

from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['STSong'] # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
print('start')
data = pd.read_csv('F:/研究生学习/研究/数据修复/'+str(lidu)+'分钟聚合数据/中环线圈数据_11'+str(startday).zfill(2)+'.csv')
for i in range(startday+1,endday+1):
    data = data.append(pd.read_csv('F:/研究生学习/研究/数据修复/'+str(lidu)+'分钟聚合数据/中环线圈数据_11'+str(i).zfill(2)+'.csv'))
    
data['det'] = data['did'].apply(lambda x:x.split('(')[0])
dataori = data.copy()  

# data = pd.DataFrame(columns = dataori.columns)

# for i in maindet:
#     data = data.append(dataori[(dataori['det']==i)])

# data['flow'] = np.round(data['flow'])
# data['spd'] = np.round(data['spd'])
# data['occ'] = np.round(data['occ'])
# data['laneno'] = data['did'].apply(lambda x:x.split('(')[1]).apply(lambda x:int(x.split(')')[0]))
# data = data.sort_values(by=['did', 'day', 'gindex']).reset_index(drop=True)

start


In [9]:
dataori

,did,day,gindex,flow,spd,occ,det
0,ZHNX29(1),4,0,31.0,76.129032,16.800000,ZHNX29
1,ZHNX29(1),4,1,36.0,76.722222,24.866667,ZHNX29
2,ZHNX29(1),4,2,28.0,76.750000,19.533333,ZHNX29
3,ZHNX29(1),4,3,32.0,76.187500,24.333333,ZHNX29
4,ZHNX29(1),4,4,24.0,77.541667,10.466667,ZHNX29
...,...,...,...,...,...,...,...
37147,ZHYP-WO-2(2),24,283,18.0,68.000000,1.533333,ZHYP-WO-2
37148,ZHYP-WO-2(2),24,284,29.0,37.517241,3.333333,ZHYP-WO-2
37149,ZHYP-WO-2(2),24,285,13.0,52.307692,1.400000,ZHYP-WO-2
37150,ZHYP-WO-2(2),24,286,17.0,52.000000,1.666667,ZHYP-WO-2


In [2]:
def myimpute(p,r,data1,dids,ego,name,seed):
    
    datasample = data1.copy()
    # %%保留真实数据
    lack = datasample.loc[r]
    origin = lack[lack['did']==ego].copy()
    
    # %%生成缺失数据
    datasample.loc[r,p] = np.nan
    # %%
    lanes = list(set(datasample['laneno']))
    # lanes

    # 1.移动平均
    # 先用LOCF补全缺失的数据

    temp = datasample[['did', 'day','min', p]][datasample['did']==ego].copy().sort_values(by=['did','day','min'])
    if np.isnan(temp[p]).any(axis=0):
        res = locf(temp[p].values.reshape(-1, 1), axis=1)
        temp[p] = np.round(res, 1)  
    else:
        temp[p] = np.round(temp[p], 1)
    
    allres = temp
        
    for n in [3,5,7]:  # 窗口大小
        for i in range(n//2):
            temp = allres[['did', 'day', 'min']].loc[allres.index.intersection(r)].reset_index(drop=True)
            temp['min'] = temp['min'] - 1 - i
            a1 = temp[temp['min'] < 0].index
            temp['day'][temp['min'] < 0] = temp['day'][temp['min'] < 0] - 1
            temp['min'] = temp['min'] % (1440/lidu)
            temp = pd.merge(temp, allres, on=['did', 'day', 'min'], how='left')
            temp.loc[a1, 'min'] = temp.loc[a1, 'min'] - (1440/lidu)
            temp['min'] = temp['min'] + i + 1
            temp.loc[a1, 'day'] = temp.loc[a1, 'day'] + 1

            if i == 0:
                res = temp[p].values.reshape(-1, 1)
            else:
                res = np.hstack([res, temp[p].values.reshape(-1, 1)])
                
        for i in range(n//2):
            temp = allres[['did', 'day', 'min']].loc[allres.index.intersection(r)].reset_index(drop=True)
            temp['min'] = temp['min'] + 1 + i
            a1 = temp[temp['min'] >= (1440/lidu)].index
            temp['day'][temp['min'] >= (1440/lidu)] = temp['day'][temp['min'] >= (1440/lidu)] + 1
            temp['min'] = temp['min'] % (1440/lidu)
            temp = pd.merge(temp, allres, on=['did', 'day', 'min'], how='left')
            temp.loc[a1, 'min'] = (1440/lidu) - temp.loc[a1, 'min']
            temp['min'] = temp['min'] - i - 1
            temp.loc[a1, 'day'] = temp.loc[a1, 'day'] - 1

            res = np.hstack([res, temp[p].values.reshape(-1, 1)])

        origin['ma' + str(n)] = np.round(np.nanmean(res, axis=1), 1)


    
    
    loops = dids.copy()
    loops.remove(ego)

    # 4.空间关联检测器修复
    from sklearn.experimental import enable_iterative_imputer  # noqa
    # now you can import normally from sklearn.impute
    from sklearn.impute import IterativeImputer
    estimator = LinearRegression()

    temp = datasample[['det', 'day', 'min', p]][datasample['did'] == ego].copy()
    temp['index'] = temp.index
    temp = temp.rename(columns={p: ego}).reset_index(drop=True)

    for i in range(len(loops)):
        did = loops[i]
        temp0 = datasample[['det', 'day', 'min', p]][datasample['did'] == did].copy()    
        temp = pd.merge(temp, temp0[['day','min',p]], on=['day', 'min'], how='left')
        temp = temp.rename(columns={p: did}).reset_index(drop=True)

    estimator = LinearRegression()
    imputer = IterativeImputer(estimator=estimator)

    itable = imputer.fit_transform(pd.concat([temp[[ego]],temp.iloc[:, 5:]],axis=1).values)

    temp[ego] = itable[:,0]
    
    temp.iloc[:, 5:] = itable[:, 1:]
    
    temp.index = temp['index']
    temp = temp.loc[temp.index.intersection(r)]
    origin['lrud'] = np.round(temp[ego].values, 1)


    
    # 5.多重插补
    temp = datasample[['did', 'day', 'min', p]][datasample['did'] == ego].copy()
    temp['index'] = temp.index
    temp = temp.rename(columns={p: ego}).reset_index(drop=True)
    
    # %%
    # explicitly require this experimental feature
    from sklearn.experimental import enable_iterative_imputer  # noqa
    # now you can import normally from sklearn.impute
    from sklearn.impute import IterativeImputer

    # %%
    estimator = LinearRegression()


    # 时间相关数据
    ## 历史时间数据
    temp['day'] = temp['day'] - 1
    temp = pd.merge(temp, datasample[['did', 'day', 'min', p]], on=['did', 'day', 'min'], how='left')
    temp = temp.rename(columns={p: 'lastday'})
    temp['day'] = temp['day'] + 1

    temp['day'] = temp['day'] + 1
    temp = pd.merge(temp, datasample[['did', 'day', 'min', p]], on=['did', 'day', 'min'], how='left')
    temp = temp.rename(columns={p: 'nextday'})
    temp['day'] = temp['day'] - 1

    ## 相邻时刻数据
    temp['min'] = temp['min'] - 1
    a1 = temp[temp['min'] < 0].index
    temp['day'][temp['min'] < 0] = temp['day'][temp['min'] < 0] - 1
    temp['min'] = temp['min'] % (1440/lidu)
    temp = pd.merge(temp, datasample[['did', 'day', 'min', p]], on=['did', 'day', 'min'], how='left')
    temp = temp.rename(columns={p: 'lastmin'})
    temp.loc[a1, 'min'] = temp.loc[a1, 'min'] - (1440/lidu)
    temp['min'] = temp['min'] + 1
    temp.loc[a1, 'day'] = temp.loc[a1, 'day'] + 1

    temp['min'] = temp['min'] + 1
    a2 = temp[temp['min'] >= (1440/lidu)].index
    temp['day'][temp['min'] >= (1440/lidu)] = temp['day'][temp['min'] >= (1440/lidu)] + 1
    temp['min'] = temp['min'] % (1440/lidu)
    temp = pd.merge(temp, datasample[['did', 'day', 'min', p]], on=['did', 'day', 'min'], how='left')
    temp = temp.rename(columns={p: 'nextmin'})
    temp.loc[set(a2), 'min'] = (1440/lidu) - temp.loc[set(a2), 'min']
    temp['min'] = temp['min'] - 1
    temp.loc[set(a2), 'day'] = temp.loc[set(a2), 'day'] - 1


    # 空间相关数据
    for i in range(len(loops)):
        did = loops[i]
        temp0 = datasample[['det', 'day', 'min', p]][datasample['did'] == did].copy()    
        temp = pd.merge(temp, temp0[['day','min',p]], on=['day', 'min'], how='left')
        temp = temp.rename(columns={p: did}).reset_index(drop=True)    
        
    estimator = LinearRegression()
    imputer = IterativeImputer(estimator=estimator)

    itable = imputer.fit_transform(pd.concat([temp[[ego]],temp.iloc[:, 5:]],axis=1).values)
    temp[ego] = itable[:,0]
    temp.iloc[:, 5:] = itable[:, 1:]

    temp.index = temp['index']
    temp = temp.loc[temp.index.intersection(r)]
    origin['mice'] = np.round(temp[ego].values, 1)


    # 6.随机森林
    # %%
    ##scaled
    
    
    
    if not os.path.exists('F:/研究生学习/研究/数据修复/'+str(lidu)+'分钟聚合数据'+ego+'_ZHNX29/seed='+str(seed)):
        os.makedirs('F:/研究生学习/研究/数据修复/'+str(lidu)+'分钟聚合数据'+ego+'_ZHNX29/seed='+str(seed))
        
#     origin.to_csv('F:/研究生学习/研究/数据修复/'+str(lidu)+'分钟聚合数据/结果0512_'+str(startday).zfill(2)+'-'+str(endday).zfill(2)+'/修复效果0128'+p+'_'+name+'.csv')      
    origin.to_csv('F:/研究生学习/研究/数据修复/'+str(lidu)+'分钟聚合数据'+ego+'_ZHNX29/seed='+str(seed)+'/修复效果0128'+p+'_'+name+'.csv')

    originzhwx342 = origin[origin['did']==ego]
    print(p,missing_rate)
    for i in ['ma5','expo0.8', 'lane', 'lrud', 'mice','RF']:
        print(np.round(pow(pow(abs(originzhwx342[i]-originzhwx342[p]),2).sum()/len(originzhwx342),0.5),2),np.round((abs(originzhwx342[i]-originzhwx342[p])/originzhwx342[p]).mean()*100,2))

## 综合场景1和场景2一起运行

In [17]:
ps=['spd']
ego = "ZHNX30(2)"
# dids = [
# "ZHNX31(1)",
# "ZHNX31(2)",
# "ZHNX31(3)",
# "ZHNX31(4)",
# "ZHNX31(5)",
# "ZHNX30(1)",
# "ZHNX30(2)",
# "ZHNX30(3)",
# "ZHNX30(4)",
# "ZHNX29(1)",
# "ZHNX29(2)",
# "ZHNX29(3)",
# "ZHNX29(4)",
# ]

data = pd.DataFrame(columns = dataori.columns)
for i in maindet:  
    data = data.append(dataori[(dataori['det']==i)])   # data是3个断面
data['flow'] = np.round(data['flow'])
data['spd'] = np.round(data['spd'])
data['occ'] = np.round(data['occ'])
data['laneno'] = data['did'].apply(lambda x:x.split('(')[1]).apply(lambda x:int(x.split(')')[0]))
data = data.sort_values(by=['did', 'day', 'gindex']).reset_index(drop=True)
        
# datasample是1个断面
datasample = data[data['det']==maindet[1]].reset_index(drop=True)

            
list1 = list(datasample['did'])
maindid = list(set(list1))
maindid.sort(key=list1.index)

list2 = list(datasample['laneno'])
lanes = list(set(list2)) 
lanes.sort(key=list2.index)

# dids = maindid
# loops = dids.copy()
# loops.remove(ego)



for seed in range(0,10):
    # 场景1
    missing_rates = [0.05,0.15,0.3]
    for missing_rate in missing_rates:    
        np.random.seed(seed)
        data1 = datasample.copy()
        random_mat = np.random.random(size=np.int(data1.shape[0]/1))     
        binary_mat = np.round(random_mat+0.5-missing_rate)
        binary_mat = np.repeat(binary_mat, 1)   # 随机缺失T取1，连续缺失T取。。。?
        r = np.where((data1[ps[0]].values!=0)&(binary_mat==0))[0].tolist() #缺失数据位置
        print('缺失数据条数：',len(r))

        for p in ps:
            myimpute(p,r,data1,dids,str(missing_rate),seed)
    
    

    # 场景2
    missing_rates = [int(10/lidu),int(20/lidu),int(30/lidu)]
    for missing_rate in missing_rates:   
        np.random.seed(seed)
        data1 = datasample.copy()
        
        random_mat1 = (np.random.rand(data1.shape[0]) > (0.15/missing_rate)).astype(int)  # 288*12*7 其中288为一天工288个5分钟，12个检测器，7天  3000/24192约为12.5%
        rhead = np.where(random_mat1==0)[0].tolist()
        
        r1 = []
        list1 = list(set(rhead))
        for i in list1:
            for j in range(missing_rate):
                r1.append(i+j)
        r1 = set(r1)
        r2 = set()
        if len(r1) < data1.shape[0] * 0.15:
            np.random.seed(seed+100)
            random_mat2 = np.random.random(size=np.int(data1.shape[0]/1))     
            binary_mat2 = np.round(random_mat2 + 0.5-(0.15-len(r1)/data1.shape[0]))
            r2 = set(np.where(binary_mat2==0)[0].tolist())

        r = list(set(r1.union(r2)))
        r.sort()
  
        print('缺失数据条数：',len(r))
        for p in ps:
            myimpute(p,r,data1,dids,str(missing_rate),seed)

缺失数据条数： 11542


<ipython-input-9-636cd72c6a0a>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] < 0] = temp['day'][temp['min'] < 0] - 1
<ipython-input-9-636cd72c6a0a>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] >= (1440/lidu)] = temp['day'][temp['min'] >= (1440/lidu)] + 1
<ipython-input-9-636cd72c6a0a>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] < 0] = temp['day'][tem

spd 2
6.48 8.6
8.12 10.69
8.92 14.56
4.16 4.53
4.08 4.39
3.68 3.16
缺失数据条数： 11489


<ipython-input-9-636cd72c6a0a>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] < 0] = temp['day'][temp['min'] < 0] - 1
<ipython-input-9-636cd72c6a0a>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] >= (1440/lidu)] = temp['day'][temp['min'] >= (1440/lidu)] + 1
<ipython-input-9-636cd72c6a0a>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] < 0] = temp['day'][tem

spd 6
8.15 11.08
9.07 12.13
9.63 15.81
3.34 4.01
3.36 4.05
2.75 2.78
缺失数据条数： 11473


...

spd 6
8.51 13.11
9.46 14.36
8.99 13.99
3.61 4.08
3.59 4.05
3.27 2.91
缺失数据条数： 11485


<ipython-input-9-636cd72c6a0a>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] < 0] = temp['day'][temp['min'] < 0] - 1
<ipython-input-9-636cd72c6a0a>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] >= (1440/lidu)] = temp['day'][temp['min'] >= (1440/lidu)] + 1
<ipython-input-9-636cd72c6a0a>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] < 0] = temp['day'][tem

spd 12
10.73 15.76
11.13 16.42
8.8 13.36
3.54 3.89
3.55 3.89
3.55 2.95
缺失数据条数： 11449


<ipython-input-9-636cd72c6a0a>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] < 0] = temp['day'][temp['min'] < 0] - 1
<ipython-input-9-636cd72c6a0a>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] >= (1440/lidu)] = temp['day'][temp['min'] >= (1440/lidu)] + 1
<ipython-input-9-636cd72c6a0a>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['day'][temp['min'] < 0] = temp['day'][tem

spd 24
16.25 23.39
16.62 24.2
8.33 13.0
3.79 4.02
3.79 4.0
3.26 2.79


In [22]:
np.random.seed(0)  
random_mat1 = (np.random.rand(1000) > (0.15/missing_rate)).astype(int)  # 288*12*7 其中288为一天工288个5分钟，12个检测器，7天  3000/24192约为12.5%
rhead = np.where(random_mat1==0)[0].tolist()

In [68]:
r1 = []
list1 = list(set(rhead))
for i in list1:
    for j in range(missing_rate):
        r1.append(i+j)
r1 = set(r1)
r2 = set()
if len(r1) < data1.shape[0] * 0.15:
    np.random.seed(seed+100)
    random_mat2 = np.random.random(size=np.int(data1.shape[0]/1))     
    binary_mat2 = np.round(random_mat2 + 0.5-(0.15-len(r1)/data1.shape[0]))
    r2 = set(np.where(binary_mat2==0)[0].tolist())
    
r = list(set(r1.union(r2)))
r.sort()

NameError: name 'data1' is not defined

In [69]:
r1 = []
list1 = list(set(rhead))
for i in list1:
    for j in range(missing_rate):
        r1.append(i+j)
r1 = set(r1)
r2 = set()
if len(r1) < data1.shape[0] * 0.15:
    np.random.seed(seed+100)
    random_mat2 = np.random.random(size=np.int(1000))     
    binary_mat2 = np.round(random_mat2 + 0.5-(0.15-len(r1)/1000))
    r2 = set(np.where(binary_mat2==0)[0].tolist())
    
r = list(set(r1.union(r2)))
r.sort()

NameError: name 'data1' is not defined

In [66]:
r = list(set(r1.union(r3)))

In [67]:
r

[522,
 523,
 524,
 525,
 14,
 15,
 16,
 17,
 530,
 531,
 540,
 541,
 545,
 34,
 35,
 546,
 43,
 44,
 571,
 572,
 586,
 587,
 75,
 76,
 588,
 589,
 82,
 83,
 603,
 604,
 97,
 98,
 99,
 100,
 630,
 631,
 126,
 127,
 655,
 656,
 159,
 160,
 673,
 674,
 166,
 167,
 682,
 683,
 171,
 172,
 173,
 174,
 179,
 692,
 180,
 691,
 697,
 698,
 707,
 708,
 709,
 198,
 199,
 204,
 205,
 206,
 722,
 723,
 214,
 215,
 740,
 741,
 742,
 743,
 233,
 234,
 757,
 758,
 262,
 263,
 268,
 269,
 283,
 284,
 800,
 801,
 307,
 308,
 309,
 310,
 822,
 824,
 823,
 821,
 322,
 323,
 837,
 838,
 332,
 333,
 852,
 853,
 343,
 344,
 351,
 352,
 390,
 391,
 903,
 904,
 394,
 395,
 398,
 399,
 911,
 912,
 409,
 410,
 411,
 412,
 413,
 927,
 928,
 417,
 418,
 931,
 932,
 427,
 428,
 942,
 943,
 443,
 444,
 968,
 969,
 457,
 458,
 990,
 991,
 480,
 481,
 507,
 508]

In [65]:
r3 = set()

In [72]:
r.sort()
r

[14,
 15,
 16,
 17,
 34,
 35,
 43,
 44,
 75,
 76,
 82,
 83,
 97,
 98,
 99,
 100,
 126,
 127,
 159,
 160,
 166,
 167,
 171,
 172,
 173,
 174,
 179,
 180,
 198,
 199,
 204,
 205,
 206,
 214,
 215,
 233,
 234,
 262,
 263,
 268,
 269,
 283,
 284,
 307,
 308,
 309,
 310,
 322,
 323,
 332,
 333,
 343,
 344,
 351,
 352,
 390,
 391,
 394,
 395,
 398,
 399,
 409,
 410,
 411,
 412,
 413,
 417,
 418,
 427,
 428,
 443,
 444,
 457,
 458,
 480,
 481,
 507,
 508,
 522,
 523,
 524,
 525,
 530,
 531,
 540,
 541,
 545,
 546,
 571,
 572,
 586,
 587,
 588,
 589,
 603,
 604,
 630,
 631,
 655,
 656,
 673,
 674,
 682,
 683,
 691,
 692,
 697,
 698,
 707,
 708,
 709,
 722,
 723,
 740,
 741,
 742,
 743,
 757,
 758,
 800,
 801,
 821,
 822,
 823,
 824,
 837,
 838,
 852,
 853,
 903,
 904,
 911,
 912,
 927,
 928,
 931,
 932,
 942,
 943,
 968,
 969,
 990,
 991]

In [24]:
if len(r1) < data1.shape[0] * 0.15:
    np.random.seed(seed+100)
    random_mat2 = np.random.random(size=np.int(data1.shape[0]/1))     
    binary_mat2 = np.round(random_mat2 + 0.5-(0.15-len(r1)/data1.shape[0]))
    r2 = set(np.where(binary_mat2==0)[0].tolist())

SyntaxError: invalid syntax (<ipython-input-24-a6ccfc46cd0d>, line 1)

In [58]:
if len(r1) < 1000 * 0.15:
    np.random.seed(100)  
    random_mat2 = np.random.random(size=np.int(1000))     
    binary_mat2 = np.round(random_mat2 + 0.5-(0.15-len(r1)/1000))
    r2 = set(np.where(binary_mat2==0)[0].tolist())

In [59]:
r2

{4, 26, 123, 397, 849, 964}

In [60]:
r1

{14,
 15,
 16,
 17,
 34,
 35,
 43,
 44,
 75,
 76,
 82,
 83,
 97,
 98,
 99,
 100,
 126,
 127,
 159,
 160,
 166,
 167,
 171,
 172,
 173,
 174,
 179,
 180,
 198,
 199,
 204,
 205,
 206,
 214,
 215,
 233,
 234,
 262,
 263,
 268,
 269,
 283,
 284,
 307,
 308,
 309,
 310,
 322,
 323,
 332,
 333,
 343,
 344,
 351,
 352,
 390,
 391,
 394,
 395,
 398,
 399,
 409,
 410,
 411,
 412,
 413,
 417,
 418,
 427,
 428,
 443,
 444,
 457,
 458,
 480,
 481,
 507,
 508,
 522,
 523,
 524,
 525,
 530,
 531,
 540,
 541,
 545,
 546,
 571,
 572,
 586,
 587,
 588,
 589,
 603,
 604,
 630,
 631,
 655,
 656,
 673,
 674,
 682,
 683,
 691,
 692,
 697,
 698,
 707,
 708,
 709,
 722,
 723,
 740,
 741,
 742,
 743,
 757,
 758,
 800,
 801,
 821,
 822,
 823,
 824,
 837,
 838,
 852,
 853,
 903,
 904,
 911,
 912,
 927,
 928,
 931,
 932,
 942,
 943,
 968,
 969,
 990,
 991}

In [61]:
r = list(set(r1.union(r2)))

In [62]:
len(r)

149

In [20]:
binary_mat2

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [19]:
r2

[]

In [15]:
(1-921/1000)*2

0.15799999999999992

In [2]:
import numpy as np

In [3]:
missing_rate = 2